# MPC SBN Replica on BigQuery

We've built a library called `mpcq` that was designed to interact at first with subscription the Small Bodies Node Postgres MPC databases. We've since ported those databases to Google BigQuery for faster queries.

Contents
- [Installation](#installation)
- [Querying for Orbits](#querying-for-orbits)
- [Querying for Observations](#querying-for-observations)
- [Crossmatching against MPC Observations](#crossmatching-against-mpc-observations)
- [Submission Information & Histories](#submission-information-&-histories)

## Installation

`mpcq` can be installed using pip:

```bash
pip install mpcq
```

Since we are working in the adam_core repository, let's add mpcq to the venv using pdm:

```bash
pdm add mpcq
```

#### Authenticating with Google 

You'll need to authenticate with Google to use the BigQuery client. See the mpcq documentation for more details.

```bash
gcloud auth application-default login
```

In [1]:
from mpcq.client import BigQueryMPCClient
# add installation

# Initialize client with your subscribed dataset IDs
client = BigQueryMPCClient(
   dataset_id="moeyens-thor-dev.mpc_sbn_aurora",
   views_dataset_id="moeyens-thor-dev.mpc_sbn_aurora_views",
   project="moeyens-thor-dev"
)

## Querying for Orbits

Let's start by querying for orbits. We'll use the `query_orbits` method to get the orbits for a list of provisional designations.

In [ ]:
# Fetch orbits for one or more objects
orbits = client.query_orbits(["2013 RR165", "2024 YR4", "A899 OF"])

# Basic analysis
print(f"Number of orbits: {len(orbits)}")

# You can view the data and get it as a pandas DataFrame
orbits.to_dataframe()

## Querying for Observations

We can also grab observations for a given list of provisional designations. One detail about the database structure is that observation of any one object can fall under any of its different provisional designations. We've taken a lot of care to ensure that you get all of the observations for an object, no matter which provisional designation it falls under and which one you query.

In [ ]:
observations = client.query_observations(["A899 OF"])
observations.to_dataframe()

## Crossmatching against MPC Observations

In some cases, especially working with archival data or in instances where you've decided to re-run source extraction pipelines, you may want to crossmatch your observations against the MPC observations. This is straightforward to do with the `cross_match_observations` method.

In [ ]:
from adam_core.observations import ADESObservations
from adam_core.time import Timestamp

input_observations = ADESObservations.from_kwargs(
    # These are the only required columns for the cross-match
    obsSubID=["1234567890", "1234567891"],
    obsTime=Timestamp.from_iso8601(['2011-01-30T11:15:25.920', '2011-01-30T11:37:22.656'], scale="utc"),
    ra=[123.884679, 123.880767],
    dec=[19.820047, 19.820603],
    stn=["F51", "F51"],
    astCat=["Gaia2", "Gaia2"],
    mode=["CCD", "CCD"],
)

# Now you can cross-match the observations
matched = client.cross_match_observations(input_observations)
matched.to_dataframe()

## Submission Information & Histories

We've also added a method to query the submission information and histories for a given provisional designations. These components are still under development as we build out our submission management system.


In [ ]:
# Get submission status
observation_status = client.query_submission_info(["2025-04-02T14:52:40.526_0000Gm0Q"])
observation_status.to_dataframe()

In [ ]:
# Get submission history
history = client.query_submission_history(["2024 YR4"])
history.to_dataframe()